In [1]:
import argparse
import configparser
import os
from datetime import datetime
from pyspark import StorageLevel
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import udf, col, from_unixtime, row_number
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, dayofweek
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType

In [2]:
# S3 bucket name
bucket_name = "sparkify-analytics-2020"
bucket_path = "s3a://{}/".format(bucket_name)

In [3]:
# AWS credentials
config = configparser.ConfigParser()
config.read('dl.cfg')
os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [4]:
# Spark session
spark = SparkSession \
        .builder \
        .appName("ETL Sparkify") \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
spark

In [5]:
# Read songplays table
songplays_path = os.path.join(bucket_path, 'songplays')
songplates_table = spark.read.parquet(songplays_path)
songplates_table.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[start_time: bigint, user_id: string, level: string, song_id: string, artist_id: string, session_id: bigint, location: string, user_agent: string, songplay_id: int, year: int, month: int]

In [6]:
# Describe songplays table
songplates_table.printSchema()
songplates_table.show(5)
print("Songplays table, number of rows: {}".format(songplates_table.count()))

root
 |-- start_time: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- songplay_id: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

+-------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+-----------+----+-----+
|   start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|songplay_id|year|month|
+-------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+-----------+----+-----+
|1543522901796|     49| paid|SOABIXP12A8C135F75|AR15DJQ1187FB5910C|      1041|San Francisco-Oak...|Mozilla/5.0 (Wind...|     